In [1]:
! pip install pytorch-lightning --quiet

     |████████████████████████████████| 675kB 5.7MB/s 
     |████████████████████████████████| 829kB 12.4MB/s 
     |████████████████████████████████| 276kB 23.6MB/s 
     |████████████████████████████████| 92kB 9.5MB/s 


In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets,transforms

import pytorch_lightning as pl

In [3]:

class LightningMNISTClassifier(pl.LightningModule):

  def __init__(self):
    super().__init__()

    # mnist images are (1, 28, 28) (channels, width, height) 
    self.layer_1 = torch.nn.Linear(28 * 28, 128)
    self.layer_2 = torch.nn.Linear(128, 256)
    self.layer_3 = torch.nn.Linear(256, 10)

  def forward(self, x):
      batch_size, channels, width, height = x.size()

      # (b, 1, 28, 28) -> (b, 1*28*28)
      x = x.view(batch_size, -1)

      # layer 1 (b, 1*28*28) -> (b, 128)
      x = self.layer_1(x)
      x = torch.relu(x)

      # layer 2 (b, 128) -> (b, 256)
      x = self.layer_2(x)
      x = torch.relu(x)

      # layer 3 (b, 256) -> (b, 10)
      x = self.layer_3(x)

      # probability distribution over labels
      x = torch.log_softmax(x, dim=1)

      return x

  def cross_entropy_loss(self, logits, labels):
    return F.nll_loss(logits, labels)

  def training_step(self, train_batch, batch_idx):
      x, y = train_batch
      logits = self.forward(x)
      loss = self.cross_entropy_loss(logits, y)
      self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
      return loss

  def validation_step(self, val_batch, batch_idx):
      x, y = val_batch
      logits = self.forward(x)
      loss = self.cross_entropy_loss(logits, y)
      self.log('val_loss', loss)

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
    return optimizer

In [4]:
class MNISTDataModule(pl.LightningDataModule):

    def __init__(self,batch_size=64):
        super().__init__()
        self.batch_size=batch_size
        self.data_dir = os.getcwd()
        self.transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])

    def prepare_data(self):
        # download
        datasets.MNIST(self.data_dir, train=True, download=True)
        datasets.MNIST(self.data_dir, train=False, download=True)

    def setup(self, stage=None):
          # transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,),(0.3801,))])
          mnist_train=datasets.MNIST(root=self.data_dir,train=True,transform=self.transform)
          mnist_test=datasets.MNIST(root=self.data_dir,train=False,transform=self.transform)

          # split dataset
          self.mnist_train,self.mnist_val=random_split(dataset=mnist_train,lengths=[55000,5000])
          self.mnist_test=datasets.MNIST(root=self.data_dir,train=False,transform=self.transform)

    def train_dataloader(self):
      mnist_train=DataLoader(dataset=self.mnist_train,batch_size=self.batch_size)
      return mnist_train

    def val_dataloader(self):
      mnist_val=DataLoader(dataset=self.mnist_val,batch_size=self.batch_size)
      return mnist_val

    def test_dataloader(self):
      mnist_test=DataLoader(dataset=self.mnist_test,batch_size=self.batch_size)
      return mnist_test

In [5]:
# data
data_module=MNISTDataModule()

# model
model=LightningMNISTClassifier()
trainer=pl.Trainer(progress_bar_refresh_rate=20,max_epochs=10)
trainer.fit(model,data_module)


GPU available: False, used: False
TPU available: None, using: 0 TPU cores


Extracting /content/MNIST/raw/train-images-idx3-ubyte.gz to /content/MNIST/raw


Extracting /content/MNIST/raw/train-labels-idx1-ubyte.gz to /content/MNIST/raw



Extracting /content/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/MNIST/raw


Extracting /content/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)



  | Name    | Type   | Params
-----------------------------------
0 | layer_1 | Linear | 100 K 
1 | layer_2 | Linear | 33.0 K
2 | layer_3 | Linear | 2.6 K 
-----------------------------------
136 K     Trainable params
0         Non-trainable params
136 K     Total params


1

In [ ]:
# TensorBoard
%load_ext tensorboard
%tensorboard --logdir lightning_dogs/